<a href="https://colab.research.google.com/github/countifs/youtube_playlist_crawling/blob/main/%EC%9C%A0%ED%8A%9C%EB%B8%8C_%EC%9E%AC%EC%83%9D%EB%AA%A9%EB%A1%9D_%ED%81%AC%EB%A1%A4%EB%A7%81(%EA%B5%AC%EA%B8%80%EC%BD%94%EB%9E%A9)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **▼ 유튜브 재생목록 크롤링 (구글 코랩 버젼)**

In [11]:
# url 주소 입력
url = input('>>>> URL 주소소를 입력 : ')

# 재생목록 제목에서 제거할 텍스트(문자)를 입력
replace_text  = input('>> 제거할 텍스트를 입력하세요 : ')

# 모듈 설치하기
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

# 웹 브라우저 작동을 위한 라이브러리
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys

# 웹브라우저 작동을 기다리기 위한 라이브러리
import time
import random

# 시간 관련 라이브러리
from datetime import datetime, timedelta
from pytz import timezone

# IPython
from IPython.display import display

# 경고 무시
import warnings
warnings.filterwarnings(action='ignore')

# 데이터프레임 및 CSV 파일 저장을 위한 라이브러리
import pandas as pd

# 데이터프레임 출력
from tabulate import tabulate

# 크롬드라이버 option설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 브라우저 생성
driver = webdriver.Chrome('chromedriver', options=options)

# 웹사이트 열기
driver.get(url)

>>>> URL 주소소를 입력 : https://www.youtube.com/playlist?list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X
>> 제거할 텍스트를 입력하세요 : 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa

In [15]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'lxml')

# 파싱정보 가져오기
parsing = soup_source.find_all('a', class_ = 'yt-simple-endpoint style-scope ytd-playlist-video-renderer')
video_time = soup_source.find_all('span', class_ ='style-scope ytd-thumbnail-overlay-time-status-renderer') #검색했을 때 검색숫자가 안맞아서 확인이 필요함

# 파싱정보 정리하기
name_list = []
url_list = []
time_list = []

for i in range(len(parsing)):
    name_list.append(parsing[i].text.strip())
    main = 'https://www.youtube.com'
    sub = parsing[i].get('href')  
    url_list.append(f'{main}{sub}')
    time_list.append(video_time[i].text.strip())

    
playlist = {
    'Title' : name_list,
    '시간' : time_list,
    '확인' : '□',    
    'No.' : range(1, len(parsing)+1),
    'URL' : url_list, 
}

playlist = pd.DataFrame(playlist)
playlist['Title'] = playlist['Title'].apply(lambda x: x.replace(replace_text,'').strip())
playlist.style.hide_index()    

Title,시간,확인,No.,URL
[Power BI] 0. 강의소개,5:43,□,1,https://www.youtube.com/watch?v=90iSaQvsDNk&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=1
[Power BI] 1. 빅데이터와 시각화,7:56,□,2,https://www.youtube.com/watch?v=Fh01tUqKbAw&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=2
[Power BI] 2. PowerBI,18:35,□,3,https://www.youtube.com/watch?v=yvUPS_RMlOI&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=3
[Power BI] 3. 강의준비하기,7:23,□,4,https://www.youtube.com/watch?v=GIlUHPz8RCQ&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=4
[Power BI] 4. 데이터 로드 및 편집,23:22,□,5,https://www.youtube.com/watch?v=dBWa2ZbJJAg&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=5
[Power BI] 5. 데이터시각화1,25:29,□,6,https://www.youtube.com/watch?v=K3S6Y_6WW0U&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=6
[Power BI] 6. 데이터시각화2,28:47,□,7,https://www.youtube.com/watch?v=cRZ6hXF6ziQ&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=7
[Power BI] 7. ppt를 활용한 디자인,6:58,□,8,https://www.youtube.com/watch?v=EqgNv-6CRpY&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=8
[Power BI] 8. 게시,8:30,□,9,https://www.youtube.com/watch?v=d-z8sQUovuA&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=9
[Power BI] 9. 모바일레이아웃,9:43,□,10,https://www.youtube.com/watch?v=YimJ8MOo3xY&list=PLjh1hlmDSDkd6_rWOjCigWKy7zzUDNj-X&index=10
